### Project 1

We need to design an dimplement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

In [29]:
import numbers
import itertools
from datetime import timedelta

class TimeZone:
    def __init__(self, name: str, offset_hours: int, offset_minutes: int):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('TimeZone name cannot be empty..')
        
        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hours must be an integer.')
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minutes must be an integer.')
        
        if abs(offset_minutes) > 59:
            raise ValueError('Must be inferior than 60.')
        
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')

        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def name(self):
        return self._name
    
    @property
    def offset(self):
        return self._offset
    
    def __eq__(self, other):
        return (isinstance(other, TimeZone)
                and self.name == other.name
                and self._offset_hours == other._offset_hours
                and self._offset_minutes == other._offset_minutes)
    
    def __repr__(self):
        return (f"TimeZone(name='{self.name}, "
                f"offset_hours={self._offset_hours}, "
                f"offsett_minutes={self._offset_minutes})")

In [23]:
timezone = TimeZone('MAR', +2, -20)

In [24]:
from datetime import datetime

dt = datetime.utcnow()
print(dt)
print(dt + timezone.offset)

2023-04-22 18:50:53.631742
2023-04-22 20:30:53.631742


In [62]:
from datetime import datetime
from collections import namedtuple

Confirmation = namedtuple('Confirmation', 'account_number transaction_code transaction_id time_utc time')

In [75]:
class Account:
    transaction_counter = itertools.count(100)
    _interest_rate = 0.5

    _transaction_codes = {
        'deposit': 'D',
        'withdraw': 'W',
        'interest': 'I',
        'rejected': 'X'

    }

    def __init__(self, 
                 account_number, 
                 first_name, 
                 last_name,
                 initial_balance=0, 
                 timezone=None):
        
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name

        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone

        self._balance = float(initial_balance)

    @property
    def account_number(self):
        return self._account_number
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self.validate_and_set_name('_first_name', value, 'First Name')
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self.validate_and_set_name('_last_name',value, 'Last Name')

    @property
    def balance(self):
        return self._balance

    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Must be a valid TimeZone.')
        self._timezone = value

    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Please enter a real number.')
        
        if value < 0:
            raise ValueError('Please enter a positive number.')
        
        cls._interest_rate = value

    def generation_confirmation_code(self, transaction_code):
        dt_str = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        return f'{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_counter)}'


    def validate_and_set_name(self, attr_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f'You must enter your {field_title}.')
        setattr(self, attr_name, value)

    @staticmethod
    def validate_real_number(value, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError('Please give a real number.')
        if min_value is not None and value < min_value:
            raise ValueError(f'Value must be at least ')
    
    @staticmethod
    def parse_confirmation_code(confirmation_code, prefered_timezone=None):
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            raise ValueError('Invalid confirmation code.')
        
        transaction_code, account_number, raw_utc, transaction_id = parts

        dt_utc = datetime.strptime(raw_utc, '%Y%m%d%H%M%S')

        if prefered_timezone is None:
            prefered_timezone = TimeZone('UTC', 0, 0)

        if not isinstance(prefered_timezone, TimeZone):
            raise ValueError('Invalid timezone.')
        
        dt_preferred = dt_utc + prefered_timezone.offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({prefered_timezone.name})"

        return Confirmation(account_number, transaction_code, transaction_id, dt_utc.isoformat(), dt_preferred_str)
    

    def deposit(self, value):
        if not isinstance(value, numbers.Real) or value <= 0:
                raise ValueError('Enter a positive number please.')
        
        transaction_code = Account._transaction_codes['deposit']
        conf_code = self.generation_confirmation_code(transaction_code)
        self._balance += value
        return conf_code
    
    def withdraw(self, value):
        
        accepted = False
        if self.balance - value < 0:
            transaction_code = Account._transaction_codes['rejected']
        else:
            accepted = True
            transaction_code = Account._transaction_codes['withdraw']
        
        conf_code = self.generation_confirmation_code(transaction_code)

        if accepted:
            self._balance -= value

        return conf_code
    
    def pay_interest(self):
        interest = self.balance * Account.get_interest_rate() / 100
        conf_code = self.generation_confirmation_code(self._transaction_codes['interest'])
        self._balance += interest
        return conf_code


In [76]:
a = Account(123, 'Didier', 'deschamps')


In [80]:
a.balance

100.0

In [78]:
a.deposit(100)

'D-123-20230422214822-100'

In [81]:
a.withdraw(150)

'X-123-20230422214841-101'